# Modeling

### What is model exactly??

Model in deep learning is a core component that takes input and output in a certain manner. It is like filter, or function in programming that takes several arguments and output certain values. 

This is a little complicated how to clearly define model and neural network. The relation between them is that neural network is a algorithm and model uses neural network as one of the main components to build. 

- Model is a filter that takes input and output value. 
- Neural network is an algorithm and model uses it as the main component. 

### Model workflow

1. Create model
2. Create loss function.
3. Configure model. 
4. Training model.
5. Save model(optional)
6. Evaluating model with test data.

### Transfer Learning

Transfer learning is one of the important methods that eliminate redundant training process in deep learning. Learning process often takes a lot of times and computer power when the training dataset is huge. This is like human teaches to another. 

### Transfer Learning Workflow

1. Create model from pre-trained data.
2. Training a new model. 
3. Evaluate. 

[Transfer Learning with a pretrained ConvNet](https://www.tensorflow.org/tutorials/images/transfer_learning#format_the_data)

[Transfer learning with TensorFlow Hub](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub#an_imagenet_classifier)

### Keras Applications

Keras Applications are set of models that is used for transfer learning with pre-trained weights. 

[Keras Applications](https://keras.io/api/applications/)

### MobileNet

MobileNet is a model in deep learning that is invented by Google and effectively designed for computer vision on mobile device. It is a CNN(convolutional neural network) and consumes less computational power to run. 

[MobileNet and MobileNetV2](https://keras.io/api/applications/mobilenet/)

### Development

[Transfer learning with pretrained ConvNet](https://www.tensorflow.org/tutorials/images/transfer_learning)

In [95]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import urllib3
urllib3.disable_warnings()

def create_model(summary=False):
    """Wrapper of tf.keras.models.Sequential()
    
    Arguments
    ---------
    summary : bool
        Dispaly summary data.
        
    Return
    ------
    model
    
    Example
    -------
    1. model = create_model()
    2. model = create_model(summary=True)
    """
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
    ])

    if summary:
        model.summary()
    
    return model


def create_predictions(model, log=False):
    """
    Arguments:
    ----------
    Model : tensorflow.python.keras.engine.sequential.Sequential'
    
    Return
    ------
    predictions : numpy.ndarray shape(1,10)
    """
    predictions = model(x_train[:1]).numpy()

    if log:
        print(type(predictions))
        print(predictions.shape)

    return predictions


# Define loss function.
def create_loss_function(log=False):
    """
    Return
    ------
    loss_fn : tensorflow.python.keras.losses.SparseCategoricalCrossentropy
    """
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    if log:
        print(type(loss_fn))
        print(loss_fn)

    return loss_fn


def compile_model(model, loss_fn, optimizer="adam"):
    """
    Arguments
    ---------
    Model     : tensorflow.python.keras.engine.sequential.Sequential
    loss_fn   : tensorflow.python.keras.losses.SparseCategoricalCrossentropy
    optimizer : str
    
    Return
    ------
    Model
    
    Example
    -------
    model = create_model()
    loss_fn = create_loss_function()
    compiled_model = compile_model(model, loss_fn)
    
    References
    ----------
    TensorFlow documentation : compile()
    https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile
    """    

    # Configure the model for training.
    # Doc compile(): https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile
    model.compile(optimizer='adam',
                  loss=loss_fn,
                  metrics=['accuracy'])
    
    return model


def train_model(model, x_train, y_train, optimizer="adam", epochs=5):
    """
    Arguments
    ---------
    model     : tensorflow.python.keras.engine.sequential.Sequential
    x_train   : uint8 arrays of grayscale image dataset. shape is 3 dimentional.
    y_train   : Label single array data.
    
    Return
    ------
    model : trained_model
    history : history of accuracy and loss
    
    Example
    -------
    # Prepare x_train & y_train dataset.
    model          = create_model()
    loss_fn        = create_loss_function()
    compiled_model = compile_model(model, loss_fn)
    (trained_model, history) = train_model(model, x_train, y_train)
    """
    
    print(type(x_train))
    print(x_train.shape)
    print(type(y_train))
    print(y_train.shape)

    # Training.
    # Trains the model for a fixed number of epochs (iterations on a dataset).
    # Doc fit(): https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit
    history = model.fit(x_train, y_train, epochs=epochs)
    
    return (model, history.history)


def save_weights(model, checkpoint_path="weights/checkpoint.ckpt"):
    """Save weights(checkpoint)
    
    Example
    -------
    # Prepare x_train & y_train dataset.
    model          = create_model()
    loss_fn        = create_loss_function()
    compiled_model = compile_model(model, loss_fn)
    (trained_model, history) = train_model(model, x_train, y_train)
    trained_model.save_weights("./weights/my.chkpt")
    """
    model.save_weights(checkpoint_path)
    
    
def train_model_from_checkpoint(model, test_images, train_labels, checkpoint_path="weights/checkpoint.ckpt"):
    """
    Train model with saved weights(checkpoint). 
    
    [save and load models: Tensorflow Document]
    https://www.tensorflow.org/tutorials/keras/save_and_load
    
    Arguments
    ---------
    model           : compiled model. 
    test_images     : x_train
    train_labels    : y_labels
    checkpoint_path : str
    
    Return
    ------
    model : trained model.
    history.history : dict
        history dictionary containing ["accuracy", "loss"]
    """
    # Create checkpoint load object.
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path, # Checkpoint filepath.
        verbose=1,                # Verbosity. 0 or 1.
        save_weights_only=True,   # If true, only weights file will be saved.
        period=5)

    # Train model with checkpoint.
    history = model.fit(x_train,  # Training image data.
                        y_train,  # Training label data.
                        epochs=3, # Number of times to train. 
                        callbacks=[cp_callback], # Checkpoint callback object.
                        verbose=1 # Verbosity. 0 or 1.
                       )

    return model, history.history


def mobilenet_model(image_size=160, display_summary=False):
    """Wrapper method of tf.keras.applications.MobileNetV2 to create model.
    
    Arguments
    ---------
    image_size : int
    display_summary : bool
    
    Return
    ------
    model
    
    Example
    -------
    mobilenet_model = mobilenet_model()
    """
    IMG_SHAPE = (image_size, image_size, 3)

    # Create the base model from the pre-trained model MobileNet V2.
    # The pre-trained file will be stored in ~/.keras/models/ by default.
    mobilenet_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                   include_top=False,
                                                   weights='imagenet')

    if display_summary:
        mobilenet_model.summary()
        
    return mobilenet_model

def evaluate_model(model, x_test, y_test):
    """Model evaluation
    
    Arguments
    ---------
    model : model
    x_test : numpy.ndarray
        uint8 arrays of digit labels (integers in range 0-9) with shapes (num_samples,).
    y_test : numpy.ndarray
        uint8 arrays of digit labels (integers in range 0-9) with shapes (num_samples,).

    Example
    -------
    # Prepare dataset.
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    # Create model.
    model          = create_model()
    loss_fn        = create_loss_function()
    compiled_model = compile_model(model, loss_fn)
    (trained_model, history) = train_model(model, x_train, y_train)
    
    evaluate_model(trained_model, x_test, y_test)

    References
    ----------
    If you want to know more details about x_test, y_test dataset, 
    please refer to the link.

    https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data?version=nightly#arguments
    """
    
    model.evaluate(x_test, y_test, verbose=2)


def visualize_history(history):
    """
    Visualization accuracy and loss data based on fit().
    
    Arguments
    ---------
    history : dict
        Dictionary data of hisotyr containing ["accuracy", "loss"]

    Example
    -------
    (trained_model, history) = train_model(model, x_train, y_train)
    visualize_history(history)
    """

    accuracy = history["accuracy"]
    loss = history["loss"]
    
    # Store in DataFrame for visualization. 
    iter_num = len(accuracy) + 1
    history_df = pd.DataFrame({
        "times": [i for i in range(1, iter_num)],
        "accuracy": accuracy,
        "loss": loss,
    })

    # Display table.
    display(history_df)

    # Lineplot with seaborn.
    sns.set_style("darkgrid")
    sns.lineplot(x="times", 
                 y="accuracy", 
                 data=history_df[["times", "accuracy"]]).set_title("accuracy")
    plt.show()

    sns.lineplot(x="times", 
                 y="loss", 
                 data=history_df[["times", "loss"]]).set_title("loss")
    plt.show()

In [88]:
if __name__ == "__main__":
    
    model = mobilenet_model()
    model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_21[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________